In [1]:
import mediapipe as mp
import numpy as np
import cv2

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) 
HUMAN_COLOR = (153, 187, 238)


# model here is holistic surrounding the code
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
    

def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def draw_styled_landmarks(image,results):
    # Draw face
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    
    
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    

def draw_face_styled_landmarks(image,results):
    # Draw face
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    
    
def extract_keypoints(results):
    # extract pose marks
    if results.pose_landmarks:
        pose = np.array([ [res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark ]).flatten()
    else:
        pose = np.zeros(num_pose_marks*4)
    
    # extract left hand
    if results.left_hand_landmarks:
        left_hand = np.array([ [res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark ]).flatten()
    else:
        left_hand = np.zeros(num_hand_marks*3)
        
        
    # extract right hand
    if results.right_hand_landmarks:
        right_hand = np.array([ [res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark ]).flatten()
    else:
        right_hand = np.zeros(num_hand_marks*3)
    
    return np.concatenate([pose,left_hand,right_hand])
    

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# new Technique

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)



pose_selected_landmarks = [
    [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31], # responsible for pose 
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], # left hand
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], # right hand
]


# contains the connection informations between points
possible_lines = []
possible_polygons = []


hand_polygons = [
    [0,1,5,9,13,17]
]



hand_connections = [
    (0,1),
    (1,2),
    (2,3),
    (3,4),
    (1,5),
    (0,17),
    (5,9),
    (9,13),
    (13,17),
    (5,6),
    (6,7),
    (7,8),
    (9,10),
    (10,11),
    (11,12),
    (13,14),
    (14,15),
    (15,16),
    (17,18),
    (18,19),
    (19,20),
]




for pair in hand_connections:
    possible_lines.append( (pair[0]+21,pair[1]+21) )
    possible_lines.append( (pair[0]+42,pair[1]+42) )

for lis in hand_polygons:
    left_hand = []
    right_hand=[]
    
    for point in lis:
        left_hand.append(point+21)
        right_hand.append(point+42)
    
    
    possible_polygons.append(left_hand)
    possible_polygons.append(right_hand)



# contains the actual points from the frame
# points = []

# contains the actual connections


# holistic model process image and return the results as keypoints
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results



def extract_keypoints(results):
    
    original_landmarks = [
        results.pose_landmarks,
        results.left_hand_landmarks,
        results.right_hand_landmarks
    ]
    
    outputs = []
    for shape in range(3):
        if(original_landmarks[shape]):
            lis = original_landmarks[shape].landmark
            pose = np.array([ [lis[res].x,lis[res].y] for res in pose_selected_landmarks[shape] ]).flatten()
            #pose = np.array([ [lis[res].x,lis[res].y] for res in range(len(lis)) ]).flatten()
        else:
            pose = np.zeros(len(pose_selected_landmarks[shape])*2)
        outputs.append(pose)
    return np.concatenate([outputs[0],outputs[1],outputs[2]])
            

def draw_landmark_from_array(image,keyPoints):
    points = [(0,0)]*63
    image_rows, image_cols, _ = image.shape
    
    
    for i in range(len(keyPoints)//2):
        x = keyPoints[i*2]
        y = keyPoints[i*2+1]
        if(x!=0 and y!=0): 
            landmark_px = mp_drawing._normalized_to_pixel_coordinates(x,y,
                                                       image_cols, image_rows)
            points[i] = landmark_px
            cv2.circle(image, landmark_px, 2, (255,255,255),
                     5)
    return points
    
    
            


 

In [4]:
possible_lines

[(21, 22),
 (42, 43),
 (22, 23),
 (43, 44),
 (23, 24),
 (44, 45),
 (24, 25),
 (45, 46),
 (22, 26),
 (43, 47),
 (21, 38),
 (42, 59),
 (26, 30),
 (47, 51),
 (30, 34),
 (51, 55),
 (34, 38),
 (55, 59),
 (26, 27),
 (47, 48),
 (27, 28),
 (48, 49),
 (28, 29),
 (49, 50),
 (30, 31),
 (51, 52),
 (31, 32),
 (52, 53),
 (32, 33),
 (53, 54),
 (34, 35),
 (55, 56),
 (35, 36),
 (56, 57),
 (36, 37),
 (57, 58),
 (38, 39),
 (59, 60),
 (39, 40),
 (60, 61),
 (40, 41),
 (61, 62)]

In [5]:
class Line:
    def __init__(self,x,y,thick=4):
        self.x=x
        self.y=y
        self.thick=thick
        
    def valid(self):
        return self.x!=None and self.y !=None


def draw_animated_lines(image,points):
    for line_index in possible_lines:
        x = points[line_index[0]]
        y = points[line_index[1]]
        line = Line(x,y,10)
        if line.valid():
            cv2.line(image, line.x, line.y, HUMAN_COLOR, line.thick)
        

def draw_animated_polygons(image,points):
    
    for polygon in possible_polygons:
        actual_points = []
        for point_index in polygon:
            try:
                x,y = points[point_index]
                actual_points.append([x,y])
            except:
                return 
                
        pts = np.array(actual_points,np.int32).reshape((-1, 1, 2))
            
        cv2.fillPoly(image, [pts], HUMAN_COLOR )
        
        

def generate_point(idx1,idx2):
    try:
        print(points[idx1])
        print(points[idx2])
        x1,y1 = points[idx1]
        x2,y2 = points[idx2]
        new_x = max(x1,x2) 
        new_y = max(y1,y2) 
        return [new_x,new_y],True
    except:
        return _,False
     
        

def draw_left_animated_polygons(image,points):
    
    polygon = possible_polygons[0]
    actual_points = []
    for point_index in polygon:
        try:
            x,y = points[point_index]
            actual_points.append([x,y])
        except:
            return 
    
    last,flag = generate_point(21,38)
    if flag:
        actual_points.append( last )

    pts = np.array(actual_points,np.int32).reshape((-1, 1, 2))

    cv2.fillPoly(image, [pts], HUMAN_COLOR )


In [12]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
        break
    
    frame, results = mediapipe_detection(frame, holistic)
    arr = extract_keypoints(results)
    new_frame = np.zeros(frame.shape) 
#     new_frame = frame
    
    draw_face_styled_landmarks(new_frame,results)
    points = draw_landmark_from_array(new_frame,arr)
    
    # draw_styled_landmarks(frame, results)
    draw_animated_lines(new_frame,points)
    draw_animated_polygons(new_frame,points)
    cv2.imshow("current frame",new_frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    

cap.release()    
cv2.destroyAllWindows()

In [24]:
cap.release()    
cv2.destroyAllWindows()